In [3]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pydantic import BaseModel


In [18]:
#cargar armi archivo dataset
movies_df = pd.read_csv('..\\backend\\data\\movies_final.csv')
credits_df =pd.read_csv('..\\backend\\data\\credits_final.csv')

C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_17176\3935945522.py:3: DtypeWarning: Columns (3,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  credits_df =pd.read_csv('..\\backend\\data\\credits_final.csv')


In [21]:
'''FITRO DE PRUEBA PARA PODER IMPLEMENTAR LA FUNCIÓN EN LOS ENDPOINTS'''
# Filtrar las filas del dataset credits_final para obtener los registros de los directores
directores = credits_df[credits_df['job'] == 'Director']

# Obtener los nombres únicos de los directores
nombres_directores = directores['name_crew'].unique()

# Filtrar el dataset movies_final utilizando los movies_id de los directores
peliculas_directores = movies_df[movies_df['movies_id'].isin(directores['movies_id'])]

# Mostrar la lista de directores con las películas que dirigieron
for nombre_director in nombres_directores:
    peliculas_dirigidas = peliculas_directores[peliculas_directores['movies_id'].isin(directores[directores['name_crew'] == nombre_director]['movies_id'])]
    print(f"Director: {nombre_director}")
    print("Películas dirigidas:")
    for titulo_pelicula in peliculas_dirigidas['title']:
        print(f"- {titulo_pelicula}")
    print()



Director: John Lasseter
Películas dirigidas:

Director: Joe Johnston
Películas dirigidas:
- Honey, I Shrunk the Kids

Director: Howard Deutch
Películas dirigidas:
- Grumpier Old Men
- The Odd Couple II

Director: Forest Whitaker
Películas dirigidas:

Director: Charles Shyer
Películas dirigidas:
- Father of the Bride Part II

Director: Michael Mann
Películas dirigidas:

Director: Sydney Pollack
Películas dirigidas:

Director: Peter Hewitt
Películas dirigidas:

Director: Peter Hyams
Películas dirigidas:
- Timecop
- 2010

Director: Martin Campbell
Películas dirigidas:
- GoldenEye
- The Mask of Zorro

Director: Rob Reiner
Películas dirigidas:

Director: Mel Brooks
Películas dirigidas:

Director: Simon Wells
Películas dirigidas:
- Balto
- An American Tail: Fievel Goes West

Director: Oliver Stone
Películas dirigidas:

Director: Renny Harlin
Películas dirigidas:
- Die Hard 2
- A Nightmare on Elm Street 4: The Dream Master

Director: Martin Scorsese
Películas dirigidas:
- The Color of Money



In [25]:
'''funcion de prueba para saber las peliculas rankeada antes de implementar en el endpoint'''
def get_peliculas_mas_votadas():
    # Convertir la columna "vote_count" a tipo numérico, manejando los valores no numéricos
    movies_df['vote_count'] = pd.to_numeric(movies_df['vote_count'], errors='coerce')
    # Filtrar los registros con un valor válido en la columna "vote_count"
    peliculas = movies_df[movies_df['vote_count'] > 2000]
    return peliculas[['title', 'vote_count', 'vote_average']].values.tolist()

# Ejemplo de uso
peliculas_mas_votadas = get_peliculas_mas_votadas()
for pelicula in peliculas_mas_votadas:
    print(f"Título: {pelicula[0]}")
    print(f"Cantidad de votos: {pelicula[1]}")
    print(f"Valor promedio de votaciones: {pelicula[2]}")
    print("---")




Título: Die Hard: With a Vengeance
Cantidad de votos: 2094.0
Valor promedio de votaciones: 6.9
---
Título: Star Wars
Cantidad de votos: 6778.0
Valor promedio de votaciones: 8.1
---
Título: The Lion King
Cantidad de votos: 5520.0
Valor promedio de votaciones: 8
---
Título: The Mask
Cantidad de votos: 2550.0
Valor promedio de votaciones: 6.6
---
Título: Jurassic Park
Cantidad de votos: 4956.0
Valor promedio de votaciones: 7.6
---
Título: Blade Runner
Cantidad de votos: 3833.0
Valor promedio de votaciones: 7.9
---
Título: Home Alone
Cantidad de votos: 2487.0
Valor promedio de votaciones: 7.1
---
Título: Aladdin
Cantidad de votos: 3495.0
Valor promedio de votaciones: 7.4
---
Título: Terminator 2: Judgment Day
Cantidad de votos: 4274.0
Valor promedio de votaciones: 7.7
---
Título: Batman
Cantidad de votos: 2145.0
Valor promedio de votaciones: 7
---
Título: The Silence of the Lambs
Cantidad de votos: 4549.0
Valor promedio de votaciones: 8.1
---
Título: Beauty and the Beast
Cantidad de votos:

In [15]:
movies_df.head(1)

,Unnamed: 0,movies_id,budget,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name,release_year,return
0,2,15602,0,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,...,Grumpier Old Men,6.5,92,119050.0,Grumpy Old Men Collection,"['Comedy', 'Romance']",['en'],['English'],1995,0


In [16]:
#eliminamos la columna "Unnamed: 0"
movies_df.drop("Unnamed: 0", axis=1, inplace=True)


In [17]:
movies_df.head()

,movies_id,budget,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name,release_year,return
0,15602,0,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101,...,Grumpier Old Men,6.5,92,119050.0,Grumpy Old Men Collection,"['Comedy', 'Romance']",['en'],['English'],1995,0
1,11862,0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106,...,Father of the Bride Part II,5.7,173,96871.0,Father of the Bride Collection,['Comedy'],['en'],['English'],1995,inf
2,710,58000000,en,James Bond must unmask the mysterious head of ...,14.686036,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034,130,...,GoldenEye,6.6,1194,645.0,James Bond Collection,"['Action', 'Adventure', 'Thriller']","['en', 'ru', 'es']","['English', 'Pусский', 'Español']",1995,6.07231093103448
3,21032,0,en,An outcast half-wolf risks his life to prevent...,12.140733,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,11348324,78,...,Balto,7.1,423,117693.0,Balto Collection,"['Adventure', 'Animation', 'Family']",['en'],['English'],1995,inf
4,9273,30000000,en,"Summoned from an ashram in Tibet, Ace finds hi...",8.205448,"[{'name': 'O Entertainment', 'id': 5682}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-11-10,212385533,90,...,Ace Ventura: When Nature Calls,6.1,1128,3167.0,Ace Ventura Collection,"['Adventure', 'Comedy', 'Crime']",['en'],['English'],1995,7.07951776666667


In [18]:
# Obtener información sobre las columnas y los tipos de datos
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2578 entries, 0 to 2577
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movies_id             2578 non-null   int64  
 1   budget                2578 non-null   int64  
 2   original_language     2578 non-null   object 
 3   overview              2578 non-null   object 
 4   popularity            2578 non-null   object 
 5   production_companies  2578 non-null   object 
 6   production_countries  2578 non-null   object 
 7   release_date          2578 non-null   object 
 8   revenue               2578 non-null   object 
 9   runtime               2578 non-null   object 
 10  status                2578 non-null   object 
 11  tagline               2578 non-null   object 
 12  title                 2578 non-null   object 
 13  vote_average          2578 non-null   object 
 14  vote_count            2578 non-null   object 
 15  collection_id        

In [19]:
# Obtener estadísticas descriptivas de las columnas numéricas
movies_df.describe()

,movies_id,budget,collection_id
count,2578.000000,2.578000e+03,2578.000000
mean,52037.655159,2.229849e+07,165643.016059
std,79760.329004,4.482319e+07,139940.209134
min,11.000000,0.000000e+00,0.000000
25%,9823.500000,0.000000e+00,47627.000000
50%,19114.000000,8.159740e+05,123255.000000
75%,50212.250000,2.100000e+07,269098.000000
max,447113.000000,3.800000e+08,479888.000000


In [20]:
#verificar valores faltantes
valores_faltantes = movies_df.isnull().sum()
print(valores_faltantes)

movies_id               0
budget                  0
original_language       0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
collection_id           0
collection_name         0
genres                  0
iso_639_1               0
name                    0
release_year            0
return                  0
dtype: int64


In [21]:
# Eliminar filas con valores faltantes
movies_df.dropna(axis=0, inplace=True)

# Eliminar columnas con valores faltantes
movies_df.dropna(axis=1, inplace=True)

In [22]:
movies_df.head(2)

,movies_id,budget,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name,release_year,return
0,15602,0,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101,...,Grumpier Old Men,6.5,92,119050.0,Grumpy Old Men Collection,"['Comedy', 'Romance']",['en'],['English'],1995,0
1,11862,0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106,...,Father of the Bride Part II,5.7,173,96871.0,Father of the Bride Collection,['Comedy'],['en'],['English'],1995,inf


In [38]:
directors_merged = pd.merge(credits_df, movies_df[['movies_id', 'title']], on='movies_id', how='inner')
directors_df = directors_merged[(directors_merged['department'] == 'Directing') & (directors_merged['job'] == 'Director')]
directors_grouped = directors_df.groupby('name_crew')['title'].apply(list).reset_index(name='directed_movies')
unique_directors = directors_grouped['name_crew'].unique()
print(unique_directors)



['Agnieszka Holland' 'Alan Mehrez' 'Alan Myerson' 'Alan Zaslove'
 'Albert Pyun' 'Alex Proyas' 'Alfred Hitchcock' 'Andrew Davis'
 'Andrew Morahan' 'Anne Goursaud' 'Arlene Sanford' 'Art Stevens'
 'Barry Cook' 'Barry Sonnenfeld' 'Ben Sharpsteen' 'Bernard Rose'
 'Betty Thomas' 'Bill Condon' 'Bobby Farrelly' 'Brad Silberling'
 'Brett Leonard' 'Brett Ratner' 'Brian De Palma' 'Brian Gibson'
 'Brian Henson' 'Brian Levant' 'Bruce Pittman' 'Bruce Sinofsky'
 'Bryan Forbes' 'Bryan Spicer' 'Charles A. Nichols' 'Charles Barton'
 'Charles Martin Smith' 'Charles Shyer' 'Chris Buck' 'Chris Columbus'
 'Chris Noonan' 'Chris Walas' 'Chris Weitz' 'Christian Duguay'
 'Christopher Cain' 'Christy Cabanne' 'Chuck Parello' 'Chuck Russell'
 'Clay Borris' 'Clyde Geronimi' 'Dale Rosenbloom' 'Damiano Damiani'
 'Dan Kuenster' 'Daniel Myrick' 'Daniel Petrie' 'Danny Boyle'
 'Danny Cannon' 'Danny Steinmann' 'David Carson' 'David Cronenberg'
 'David Fincher' 'David Hand' 'David Kellogg' 'David Lowell Rich'
 'David Price

In [12]:
#guardamos esos datos
movies_df.to_csv('..\\backend\\data\\movies_final.csv')

In [18]:
# Preprocesar los datos y obtener la matriz de características TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
movies_df['overview'] = movies_df['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies_df['overview'])

In [19]:
# Calcular la similitud coseno entre las películas
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
# Función para obtener las películas recomendadas
def obtener_recomendaciones(titulo, cosine_sim, movies_data):
    # Obtener el índice de la película según su título
    indices = pd.Series(movies_data.index, index=movies_df['title']).drop_duplicates()
    idx = indices[titulo]

    # Obtener los puntajes de similitud de la película
    scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas según los puntajes de similitud
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares
    top_indices = [i[0] for i in scores[1:6]]

    # Obtener los títulos de las películas recomendadas
    recomendaciones = movies_df['title'].iloc[top_indices].tolist()

    return recomendaciones

In [25]:
# Ahora haceso el  uso de la función
titulo_pelicula = input("Ingrese el título de la película: ")
if titulo_pelicula in movies_df['title'].values:
    recomendaciones = obtener_recomendaciones(titulo_pelicula, cosine_sim, movies_df)
    print(f"Películas recomendadas para '{titulo_pelicula}':")
    for pelicula in recomendaciones:
        print(pelicula)
else:
    print("La película no fue encontrada en la base de datos.")

Películas recomendadas para 'Father of the Bride Part II':
Ace Ventura: Pet Detective
Ong-Bak: The Thai Warrior
Sinbad and the Eye of the Tiger
To Trap a Spy
Rambo: First Blood Part II


In [23]:
print(movies_df['title'])

2                                         Grumpier Old Men
4                              Father of the Bride Part II
9                                                GoldenEye
12                                                   Balto
18                          Ace Ventura: When Nature Calls
                               ...                        
45342    Schoolgirl Report Part 5: What All Parents Sho...
45343    Schoolgirl Report Part 4: What Drives Parents ...
45360                                        Descendants 2
45457                           Frankenstein Created Woman
45462                                     Carry On Camping
Name: title, Length: 2578, dtype: object


In [27]:
list_movies = movies_df['title']
list_movies.head()

2                   Grumpier Old Men
4        Father of the Bride Part II
9                          GoldenEye
12                             Balto
18    Ace Ventura: When Nature Calls
Name: title, dtype: object

In [28]:
list_movies.to_csv('..\\backend\\data\\list_movies.csv')

In [16]:
list_cast = pd.read_csv('..\\backend\\data\\credits_final.csv')

C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_17176\753471148.py:1: DtypeWarning: Columns (3,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  list_cast = pd.read_csv('..\\backend\\data\\credits_final.csv')


In [17]:

list_cast = list_cast['name_crew']
list_cast.head(10)

0    John Lasseter
1    John Lasseter
2    John Lasseter
3    John Lasseter
4    John Lasseter
5    John Lasseter
6    John Lasseter
7    John Lasseter
8    John Lasseter
9    John Lasseter
Name: name_crew, dtype: object